In [1]:
import os
# Pytorch evaluates env vars on import instead of runtime so this needs to be at the top
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29500'

In [2]:
from llama.generation import Llama

In [3]:
ckpt_dir = './Meta-Llama-3-8B-Instruct/'
tokenizer_path = 'Meta-Llama-3-8B-Instruct/tokenizer.model'
ckpt_path = 'Meta-Llama-3-8B-Instruct/consolidated.00.pth'
params_path = 'Meta-Llama-3-8B-Instruct/params.json'

In [4]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

In [5]:
llama = Llama.build(ckpt_dir, tokenizer_path, max_seq_len=512, model_parallel_size=1, max_batch_size=4, seed=42)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/Users/shanewinterhalter/Projects/security_tool/_venv/lib/python3.11/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


Loaded in 40.14 seconds


In [6]:
# First generation to preload model
llama.chat_completion(
    [[{"role": "user", "content": "abcde"}]],
    max_gen_len=1,
    temperature=0,
    top_p=0.9,
)

/Users/shanewinterhalter/Projects/security_tool/llama3/llama/generation.py:218: UserWarning: The operator 'aten::isin.Tensor_Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  torch.isin(next_token, stop_tokens)


[{'generation': {'role': 'assistant', 'content': 'The'}}]

In [7]:
%timeit results = llama.chat_completion([messages], max_gen_len=10, temperature=0.7, top_p=0.9)
# 2.39s without torch compile & MPS

2.34 s ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
